In [41]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf

## DATASET 
Download from kaggle 
link : https://www.kaggle.com/mczielinski/bitcoin-historical-data?select=bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv

In [2]:
train = pd.read_csv("btc.csv")

In [3]:
train.tail(20)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
4857357,1617147660,58616.33,58616.33,58616.33,58616.33,0.000691,4.051737e+01,58616.330000
4857358,1617147720,58606.95,58620.59,58606.95,58620.59,0.003404,1.995570e+02,58618.245075
4857359,1617147780,58608.96,58608.96,58606.95,58608.38,17.443451,1.022333e+06,58608.402406
4857360,1617147840,58599.78,58600.00,58599.78,58600.00,3.000086,1.758051e+05,58599.999957
4857361,1617147900,58599.99,58600.00,58599.99,58600.00,10.412178,6.101536e+05,58599.999683
4857362,1617147960,58631.74,58699.00,58631.74,58666.54,10.421432,6.116325e+05,58689.870394
4857363,1617148020,58680.18,58702.79,58680.18,58702.79,0.144199,8.461819e+03,58681.583871
4857364,1617148080,58699.45,58699.45,58675.12,58677.70,1.165074,6.836340e+04,58677.332331
4857365,1617148140,58673.20,58673.20,58644.13,58644.13,0.843998,4.949565e+04,58644.258412
4857366,1617148200,58673.20,58689.44,58673.20,58689.44,0.576412,3.382238e+04,58677.435702


In [4]:
train.isnull().sum()

Timestamp                  0
Open                 1243608
High                 1243608
Low                  1243608
Close                1243608
Volume_(BTC)         1243608
Volume_(Currency)    1243608
Weighted_Price       1243608
dtype: int64

In [ ]:
train.count()

In [ ]:
train.info()

In [ ]:
train.describe()

In [5]:
train = train.dropna()
train.isnull().sum()

Timestamp            0
Open                 0
High                 0
Low                  0
Close                0
Volume_(BTC)         0
Volume_(Currency)    0
Weighted_Price       0
dtype: int64

In [ ]:
train.count()

In [6]:
train = train.drop(columns=['Timestamp'])
#train.set_index('Timestamp')

#train['Timestamp'] = pd.to_datetime(train['Timestamp'], unit='s')

In [7]:
train.shape

(3613769, 7)

In [ ]:
#train['Timestamp'].tail()

In [37]:
# split_date = pd.to_datetime(['2019-12-31 07:52:00'])
#train.head()
data_train = train.loc[:2312812].copy()
data_val = train.loc[2312813:2891016].copy()
data_test = train.loc[2891017:].copy()

In [19]:
data_train.count()

Open                 1272636
High                 1272636
Low                  1272636
Close                1272636
Volume_(BTC)         1272636
Volume_(Currency)    1272636
Weighted_Price       1272636
dtype: int64

In [25]:
X_train = data_train
Y_train = X_train['Close']
X_train = X_train.iloc[:-1]
Y_train = Y_train.iloc[1:]

In [26]:
X_test = data_test
Y_test = X_test['Close']
X_test = X_test.iloc[:-1]
Y_test = Y_test.iloc[1:]

In [27]:
X_val = data_val
Y_val = X_val['Close']
X_val = X_val.iloc[:-1]
Y_val = Y_val.iloc[1:]

In [28]:
X_train.shape

(1272635, 7)

In [59]:
X_test.shape

(1915091, 7)

In [29]:
Y_train.shape

(1272635,)

In [35]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

In [42]:
train_set = windowed_dataset(data_train, window_size=60, batch_size=100, shuffle_buffer=1000)

In [58]:
X_train = np.reshape(X_train, (-1, 60,1))
X_test = np.reshape(X_test, (-1, 60,1))

ValueError: cannot reshape array of size 8908445 into shape (60,1)

In [15]:
from tensorflow.keras.layers import Dense, LSTM, Flatten, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Embedding

In [48]:
#buat model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
# Second LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Third LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.5))
# Fourth LSTM layer
model.add(LSTM(units=50))
model.add(Dropout(0.5))
# The output layer
model.add(Dense(units=1))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 7, 50)             10400     
_________________________________________________________________
dropout_12 (Dropout)         (None, 7, 50)             0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 7, 50)             20200     
_________________________________________________________________
dropout_13 (Dropout)         (None, 7, 50)             0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 7, 50)             20200     
_________________________________________________________________
dropout_14 (Dropout)         (None, 7, 50)             0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 50)               

In [53]:
batch_size = 16

In [44]:
from tensorflow.keras.optimizers import Adam

In [56]:
# Compiling the RNN
model.compile(optimizer=Adam(lr=0.001),metrics=["mae"], loss='mean_absolute_error')
# Fitting to the training set
history = model.fit(X_train, Y_train, epochs=20, batch_size=batch_size,
                    validation_data=(X_val, Y_val), verbose=2, validation_split=0.1)

Epoch 1/20


ValueError: in user code:

    c:\users\user\anaconda3\envs\py37\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\users\user\anaconda3\envs\py37\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\user\anaconda3\envs\py37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\user\anaconda3\envs\py37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\user\anaconda3\envs\py37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\user\anaconda3\envs\py37\lib\site-packages\tensorf